# Welcome to the Echo package

The "echo" subpackage of murraylab_tools is designed to automate the setup of reactions with the Echo, with special functionality provided for TX-TL reactions. The EchoRun class can produce picklists for the Echo, as well as instructions for loading a source plate to go with those instructions, when appropriate. 

There are currently three main modes of operations of the Echo package, distinguished mainly by their inputs:
- **TX-TL Setup Spreadsheet**: Takes a pair of CSV spreadsheets saved from a TX-TL setup spreadsheet (version 2.X). Use this for simple TX-TL experiments, or very complex experiments that would be easier to set up in a spreadsheet than programmatically.
- **Programmatic Setup**: You can build a reaction programmatically, straight from code. Build objects representing the materials you want to use (including mixtures of things) and add them to wells on a destination plate with:
    - **`dilution_series`**: Takes two materials and an array of concentrations for each, and builds a 2D dilution series out of the two. Can also be used to set up 1D dilution series (by setting one of the materials to a dummy material and giving it the concentration list [0]).
    - **`add_material_to_well`, `add_material_to_block`, `add_material_to_all`**: Adds a single ingredient at a fixed concentration, volume, or fraction of the final reaction to either a single well, a rectangular block of wells, or all wells that the EchoRun object has used so far, respectively.
    - **`fill_well_with`,`fill_all_with`**: Whatever volume is left in either a well or every well the EchoRun object has interacted with so far, respectively, fill with a material.
    
- **Association Spreadsheet**: Takes one or more spreadsheets describing the contents of an Echo source plate, plus a simple spreadsheet describing final concentrations of the materials of those source plates in each destination. You can use this if you are more comfortable working with spreadsheets than with Python.

*A word of warning*: The quick start examples will help you jump right into setting up an experiment, but they make a number of assumptions about your experiment. Some things you'll want to check before running your own:
- **Reaction Size**: Default is 10 µL.
- **Buffer/Extract Fractions**: Defaults are 0.42/0.33 (typical for French Press extracts).
- **Buffer/Extract Aliquot Size**: Defaults are 30µL/37µL per aliquot, respectively. 
- **Master Mix Excess**: For accounting for pipetting loss. Default is 1.1 (10% excess) for `TXTLMasterMix`, and 1 (no excess) for `MixtureMaterial`.
- **Source Plate Type**: Default is 384_PP. You probably want this.
- **Source Plate Material**: Default is AQ_BP (buffer-like liquids).
- **Destination Plate Type**: Default is "Nunc_384_black_glassbottom".
- **Destination Plate Size**: The Echo package has no knowledge of the destination plate. *There is no check to keep you from defining picks off the edge of the destination plate*. Don't do that.
- **Dead Volume/Max Volume**: Default dead volume is 21 µL, including loss to meniscus. Default max volume is 65 µL. 

**Additionally, be aware** that the TX-TL setup scripts keep track of which source plate wells they've used in a .dat file. If you run those experiments repeatedly, they'll fill up the file and eventually error out when they run out of wells on the plate. You should test your code using a dummy .dat file, then switch to the one you actually want to use and run it again once you're sure your code works.

You can read more about how the echo package works in the next section, "**How it all works**", or you can skip to the example usage sections below that. For more information on tweaking settings, see **Tweaking Settings** at the end of this notebook.

See the following notebooks for other Echo setup features:
* `Reusable Plate Example`: To re-use materials stored in a fixed position on a source plate.
* `echo_dilution_series`: To make 3D (or higher-dimension) dilution series.
* `Echo Setup - salt calibration example`: An example of setting up a slightly-more-complex-than-usual TX-TL experiment with a custom master mix.

# How it all works

Every project begins with an `EchoRun` object. This object holds (nearly) all of the information about an experiment, and is ultimately responsible for coordinating plates and materials, and for actually writing picklists and experimental protocols. 

An `EchoRun` object needs a `SourcePlate` object. This object is responsible for keeping track of which wells have been used, and for assigning new wells on the source plate to materials that you'll want to transfer. It will try to assign wells in a way that keeps the same material in a contiguous block, buffered on either side by an empty well for ease of pipetting. `SourcePlate` objects are also typically associated with a .dat tracking file that lists which wells have been used on the plate. That way, you can use the same source plate over many experiments without having to manually program in which wells are forbidden. The `SourcePlate` object will read this file to learn what it has available to it, and will automatically write back to the same file whenever the EchoRun object controlling it writes a picklist.

Materials (DNA, chemicals, water, TX-TL, etc) on a source plate are represented by `EchoSourceMaterial` objects. An `EchoSourceMaterial` has a concentration, which can be of any units you want. For the most part, units are only there for human readability. Special exceptions are made for dsDNAs, which are usually kept in stocks with units of ng/uL and used in units of nM. `EchoSourceMaterial`s representing dsDNAs are created with a "length". If an `EchoSourceMaterial` has a "length" > 0, then it will assume it was a dsDNA created in units of ng/uL, and will immediately convert its concentration into nM. Anything using that material should use units of nM.

To generate an Echo picklist, you will generally need to do three things:
- Build an `EchoRun` object. This will take a `SourcePlate` object, which usually just needs to know the location of the used-well file.
- Describe the experiment. This means calling one of three things from your `EchoRun` object: 
    - `build_picklist_from_txtl_setup_csvs`, or
    - `load_source_plate` to learn what's on the source plate and `build_picklist_from_association_spreadsheet` to figure out what goes where on the destination plate, or 
    - Programmatic setup using `dilution_series`, `add_material_to_well`, `fill_well_with`, etc.
- Write the picklist, which is done with a call to `write_picklist` from your `EchoRun` object. 

For more information, see the examples below.

## TX-TL Setup Spreadsheet

### Quick Start Example

The following creates an Echo picklist and experimental protocol for a variety of fluorescent protein mixes described in "inputs/TX-TL_setup_example.xlsx".

![Example TX-TL setup spreadsheet](resources/TX-TL_setup_example_screenshot.png)

In [58]:
import murraylab_tools.echo as mt_echo
import os.path

# Names of input and output files. Check out these files for examples of input file format.
txtl_inputs  = os.path.join("txtl_setup", "inputs")
txtl_outputs = os.path.join("txtl_setup", "outputs")
stock_file  = os.path.join(txtl_inputs, "TX-TL_setup_example_stocks.csv") # Source materials
recipe_file = os.path.join(txtl_inputs, "TX-TL_setup_example_recipe.csv") # Experimental setup 
plate_file  = os.path.join(txtl_outputs, "TX-TL_setup_example_plate.dat")  # Keeps track of wells used 
output_name = os.path.join(txtl_outputs, "TX-TL_setup_example") # Output (both a picklist and a
                                                                # small protocol for building the
                                                                # source plate)

# Build an EchoRun object
txtl_plate = mt_echo.SourcePlate(filename = plate_file)
txtl_echo_calculator = mt_echo.EchoRun(plate = txtl_plate)

# Describe the experiment
txtl_echo_calculator.build_picklist_from_txtl_setup_csvs(stock_file, recipe_file)
# Write results
txtl_echo_calculator.write_picklist(output_name)

 ### More Information

The `build_picklist_from_txtl_setup_csvs` function is used for creating a picklist from a TX-TL setup spreadsheet (spreadsheet version 2.0 or later -- spreadsheets from before late 2016 will not work). You will need to feed it the names of two CSV files produced from the "recipe" sheet (the first sheet, containing explicit pipetting directions) and the "stocks" sheet (the second sheet, which details the concentrations of most of the materials used in the experiment). The "Layout" sheet is ignored. The standard workflow looks like:

* Edit your TX-TL setup Excel document (modifying the "Stocks" sheet and the "Layout" sheet, and the few cells shaded purple in the "Recipe" sheet).
* Save the "Recipe" and "Stocks" sheets from the xls/xlsx file as CSVs.
* Run `build_picklist_from_txtl_setup_csvs`, passing it the names of the two CSVs you just saved.

Reaction size and master mix excess ratio are read from the recipe spreadsheet. You probably should not mess with those settings 

Note that you *must* give each reaction a plate location, i.e. "D4" or "E07". In the Excel spreadsheet. Plate locations can be added in the "Layout" tab, and will be automatically propagated to the "Recipe" tab. 

## Programmatic construction of TX-TL Reactions

#### Quick Start Example

The following creates an Echo picklist and simple experimental protcol for a two-way dilution series of a reporter plasmid and inducer, with or without a second inducer. This code will add four things to the destination plate:
1. Two 2-dimensional dilution series of GFP plasmid on one axis and aTc on the other axis.
2. TX-TL master mix into every well.
3. Water to fill up remaining space in every well.
4. IPTG in one of the dilution series blocks, and in two negative control wells (one at 0 concentration).

You'll usually end up adding some variation of 1 (experimental conditions), 2 (a master mix), and 3 (a buffer to fill).

In [59]:
import murraylab_tools.echo as mt_echo
import os.path

# Relevant input and output files. Check these out for examples of input file format.
dilution_inputs  = os.path.join("2D_dilution_series", "inputs")
dilution_outputs = os.path.join("2D_dilution_series", "outputs")
plate_file  = os.path.join(dilution_outputs, "dilution_setup_example_plate.dat") # Keeps track of wells used
output_name = os.path.join(dilution_outputs, "dilution_setup_example") # Output (both a picklist and a
                                                               # small protocol for building the
                                                               # source plate)

# Build an EchoRun object
dilution_plate = mt_echo.SourcePlate(filename = plate_file)
dilution_echo_calculator = mt_echo.EchoRun(plate = dilution_plate)

# Set final concentrations of two materials
gfp_final_concentrations = range(0,6,1) # in nM
atc_final_concentrations = range(0,100,20) # in ng/uL

# Define reporter plasmid material
gfp_conc = 294  # Concentration in ng/uL
gfp_len  = 3202 # Size of DNA in bp
gfp = mt_echo.EchoSourceMaterial('GFP Plasmid', gfp_conc, gfp_len)

# Define inducer material
atc_conc = 1000 # Concentration in ng/uL (important that this matches the units of the final concentrations)
atc = mt_echo.EchoSourceMaterial("ATc", atc_conc)

# Define a second inducer material
iptg_conc = 500 # Concentration, this time in mM. Again, this must match any other units used for IPTG.
iptg = mt_echo.EchoSourceMaterial("IPTG", iptg_conc)

# Lay out the two dilution series
dilution_echo_calculator.dilution_series(gfp, atc, gfp_final_concentrations,
                                         atc_final_concentrations, "A1")
dilution_echo_calculator.dilution_series(gfp, atc, gfp_final_concentrations,
                                         atc_final_concentrations, "A7")

# Add the second inducer to the second dilution series.
iptg_final_conc = 5
dilution_echo_calculator.add_material_to_block(iptg, "A7", "F11", iptg_final_conc)

# Negative control wells.
dilution_echo_calculator.add_material_to_well(iptg, "G1", 0)
dilution_echo_calculator.add_material_to_well(iptg, "G7", iptg_final_conc)

# Add master mix
master_mix = mt_echo.TXTLMasterMix()
dilution_echo_calculator.add_material_to_all(master_mix) # Don't give mixtures a concentration.
                                                         # They'll figure it out.

# Fill empty space with water
water = mt_echo.EchoSourceMaterial("water", 1)
dilution_echo_calculator.fill_all_wells_with(water)

# Write results
dilution_echo_calculator.write_picklist(output_name)

Note the warnings -- a lot of mistakes you might make will cause you to pipette 0 nL at a time, so the code will warn you if you do so. In this case, those 0 volume pipetting steps are normal -- you just added a material at 0 concentration. Similar warnings will appear if you under-fill a reaction.

The final `write_picklist` command produces two files -- an Echo picklist, and a plaintext experiment file giving brief instructions for loading the source plate.

The Echo picklist (2D_dilution_series/outputs/dilution_setup_example_EchoInput.csv; the rest of the dilution series examples output to the same folder):

![Example Echo picklist](resources/echo_picklist_screenshot.png)

The plaintext experiment file (2D_dilution_series/outputs/dilution_setup_example_experiment_overview.csv):

![Experiment file](resources/echo_experiment_file_screenshot.png)

#### Adding components individually

You can also manually add a single material to a well (`add_material_to_well`) or a rectangular block of wells (`add_material_to_block`) by specifying the material (an EchoSourceMaterial object), a final concentration, and a location. For example, if we set up a dilution series using the variables above...

In [60]:
# Build an EchoRun object
dilution_plate = mt_echo.SourcePlate(filename = plate_file)
# default_master_mix = mt_echo.MasterMix(plate = dilution_plate)
dilution_echo_calculator = mt_echo.EchoRun(plate = dilution_plate)

# Define materials
# Note: Don't try to re-use an existing EchoSourceMaterial object if that object has been
#       used already in an EchoRun object that has had its write_picklist function called,
#       or you will get errors
gfp = mt_echo.EchoSourceMaterial('GFP Plasmid', gfp_conc, gfp_len)
atc = mt_echo.EchoSourceMaterial("ATc", atc_conc)

# # Plan out a dilution series
starting_well = "D2"
dilution_echo_calculator.dilution_series(gfp, atc, gfp_final_concentrations,
                                               atc_final_concentrations, starting_well)

...then we can add, say, bananas, to a 2x2 square in the top-left corner of the reaction.

In [61]:
# Bananas at 100 nM
bananas = mt_echo.EchoSourceMaterial('Cavendish', 100)
dilution_echo_calculator.add_material_to_block(bananas, 'D2', 'E3', 1)

You can also add to a single well, if you really need to.

In [62]:
old_bananas = mt_echo.EchoSourceMaterial('Gros Michel', 100)
dilution_echo_calculator.add_material_to_well(old_bananas, 'F5', 3)

If you want something to be in the final reaction but want to add it to your destination plate by hand instead of having the echo pipette it (e.g., you might not want to echo the TX-TL master mix), you can set this in the `add_material_to_block` or `add_material_to_well` functions. If you set this flag, directions to add this material manually will be added to the experiment overview file. 

In [63]:
viscous_bananas = mt_echo.EchoSourceMaterial('Viscous', 100)
dilution_echo_calculator.add_material_to_well(viscous_bananas,"E4", 5, pipette_by_hand = True)
dilution_echo_calculator.write_picklist(output_name + "_banana_case")

Notice the warnings about underfilled reactions -- this is because we're only adding a little bit of stuff to the mix, and haven't added water or any other bulk solvent. You can fill up the remainder of a well with `EchoRun.fill_well_with` or `EchoRun.fill_all_wells_with`:

In [64]:
dilution_plate = mt_echo.SourcePlate(filename = plate_file)
dilution_echo_calculator = mt_echo.EchoRun(plate = dilution_plate)

gfp = mt_echo.EchoSourceMaterial('GFP Plasmid', gfp_conc, gfp_len)
atc = mt_echo.EchoSourceMaterial("ATc", atc_conc,)

# Bananas at 100 nM
bananas = mt_echo.EchoSourceMaterial('Cavendish', 100)
dilution_echo_calculator.add_material_to_block(bananas, 'D2', 'E3', 1)

old_bananas = mt_echo.EchoSourceMaterial('Gros Michel', 100)
dilution_echo_calculator.add_material_to_well(old_bananas, 'F5', 3)

viscous_bananas = mt_echo.EchoSourceMaterial('Viscous', 100)
dilution_echo_calculator.add_material_to_well(viscous_bananas, "E4", 5, pipette_by_hand = True)


# Here's where we fill with ingredients.
water = mt_echo.EchoSourceMaterial('Water', 1)
saltwater = mt_echo.EchoSourceMaterial('Salt Water', 1)

dilution_echo_calculator.fill_well_with(water, "D2")
dilution_echo_calculator.fill_all_wells_with(saltwater)

# All reactions are filled! No warnings!
dilution_echo_calculator.write_picklist(output_name + "_fill_case")

#### Master Mixes 

There are a couple of different kind of "master mix" objects:
* `MixtureMaterial`: This represents a generic mix of liquids that will be used as a source material. 
* `TXTLMasterMix`: A subclass of `MixtureMaterial` that's specialized for TX-TL reactions. `TXTLMasterMix` objects come with built-in extract and buffer components, and have a variety of settings for tweaking the TX-TL part of an experiment.

Here's an example use of a master mix as you might use for PCR:

In [65]:
# File locations
pcr_folder = "PCR"
plate_file = os.path.join(pcr_folder, "PCR_plate.dat")
output_name = os.path.join(pcr_folder, "output", "PCR_experiment")

# EchoRun setup
source_plate = mt_echo.SourcePlate(plate_file)
pcr_echo_calculator = mt_echo.EchoRun(rxn_vol = 50000)

# Define materials
q5       = mt_echo.EchoSourceMaterial("NEB Q5 HS 2x MM", 2, units = "x")
f_primer = mt_echo.EchoSourceMaterial("SC189", 10, units = "uM")
r_primer = mt_echo.EchoSourceMaterial("SC190", 10, units = "uM")
water    = mt_echo.EchoSourceMaterial("water", 1, units = "x")
t1       = mt_echo.EchoSourceMaterial("Template 1", 110, length = 1856)
t2       = mt_echo.EchoSourceMaterial("Template 2", 205, length = 1856)
t3       = mt_echo.EchoSourceMaterial("Template 3", 89, length = 2010)
t4       = mt_echo.EchoSourceMaterial("Template 4", 100, length = 2010)
t5       = mt_echo.EchoSourceMaterial("Template 5", 25, length = 154)
templates = [t1, t2, t3, t4, t5]

# Make the master mix
master_mix = mt_echo.MixtureMaterial("Master Mix", rxn_vol = 50000, recipe_excess = 1.1)
master_mix.add_material(q5, 1, dimensionality = "final_concentration")
master_mix.add_material(f_primer, 0.5, dimensionality = "final_concentration")
master_mix.add_material(r_primer, 0.5, dimensionality = "final_concentration")
master_mix.add_material(water, 19.0/50.0, dimensionality = "final_concentration")

# Add materials to the plate
for i in range(len(templates)):
    template = templates[i]
    well = "A" + str(i+1)
    pcr_echo_calculator.add_material_to_well(template, well, 1)
pcr_echo_calculator.add_material_to_all(master_mix, 49.0/50.0)
pcr_echo_calculator.fill_all_wells_with(water)



In [66]:
pcr_echo_calculator.write_picklist(output_name)

You can find more information about how to use `Mixture` and `TXTLMasterMix` in the **Tweaking Settings** section below.

## Association Spreadsheet

### Quick Start Example

The following creates an Echo picklist for an automated PCR setup with three sets of primers to be applied individually to three different plasmids, as defined in a pair of CSV files (one defining the source plate, one describing what should go in the destination plates). 

The source plate definition file (association_list/inputs/association_source_sheet.csv):

![Source plate definition CSV](resources/source_csv_screenshot.png)

The destionation file (association_list/inputs/association_destination_sheet.csv):

![Destination file CSV](resources/destination_CSV_screenshot.png)

In [67]:
import murraylab_tools.echo as mt_echo
import os.path

# Relevant input and output files. Check these out for examples of input file format.
assoc_inputs  = os.path.join("association_list", "inputs")
assoc_outputs = os.path.join("association_list", "outputs") 
stock_file = os.path.join(assoc_inputs, 'association_source_sheet.csv')
assoc_file = os.path.join(assoc_inputs, 'association_final_sheet.csv')
assoc_name = os.path.join(assoc_outputs, 'association_example')

# Build an EchoRun object
assoc_echo_calculator = mt_echo.EchoRun(plate_log_folder = os.path.join(assoc_outputs,"source[1].txt"))
assoc_echo_calculator.rxn_vol = 50000 # PCR is large-volume!

# Define which column of the source file is what
name_col  = 'B'
conc_col  = 'C'
len_col   = 'D'
well_col  = 'A'
plate_col = 'E'

# Define the source plate based on the stock file.
assoc_echo_calculator.load_source_plate(stock_file, name_col, conc_col,
                                        len_col, well_col, plate_col)

# Build a protocol, based on the association file.
assoc_echo_calculator.build_picklist_from_association_spreadsheet(assoc_file,
                                                                  well_col,
                                                                  fill_with_water = True,
                                                                  water_name = "water")

# Write the picklist
assoc_echo_calculator.write_picklist(assoc_name)

### More Information

The `build_picklist_from_association_spreadsheet` function is used for arbitrary mappings of source plate wells to destination wells, using 1) a spreadsheet describing the contents of each source plate, and 2) a second spreadsheet describing what materials should be put in what wells, and at what concentration. You must always set up a source plate first. This should be done by calling `load_source_plate` with the name of the source plate spreadsheet and information about its organization. Alternatively, you could build your own manually. That is not recommended. 

The first 'source' spreadsheet (describing the source plate) is a CSV-format spreadsheet with, at minimum, columns with the following information. You can add any number of additional columns to the source plate spreadsheet, which will be ignored by this function. This is the spreadsheet read in with the `load_source_plate` command.
* Location: This is the well number of the material, i.e. "C4" or "E08". If the same material is found in multiple wells, it will need one row for each well.
* Name: Brief string describing the material. This name will be used in the recipe output of EchoRun. It can be any string, but be aware that the names "water" and "txtl_mm" are reserved for describing water and TX-TL master mix, respectively. For this function, that won't matter, but if you try to combine other setup commands with this one, you should avoid using "water" and "txtl_mm" as material names. 
* Concentration: If the material is dsDNA, this should be the concentration of the DNA in ng/µL. Otherwise, it should be the concentration of the material in whatever units you want.
* Length: If the material is dsDNA, this should be the number of base pairs in the DNA. Otherwise, length should be 0. This is important for correct unit usage.
* Units: Describes the units you used to specify concentration. This is purely for human readout purposes, and won't affect calculations. If you give a non-zero length, this will be ignored, and units of ng/uL will be assumed (and converted into nM).

The second 'association' spreadsheet (describing the what materials go together) is also a CSV-format spreadsheet. This spreadsheet determines what goes into the destination well. One column of the association spreadsheet determines that row's well on the destination plate. The EchoRun object will scan through every column, ignoring the well column, taking pairs of columns from left to right. There can be any number of pairs of columns; each one will cause one material to be moved from the source plate to the destination plate. The first clumn in each pair holds the name of a material. This name must exactly match one of the material names listed in the source spreadsheet, and determines where material will be taken from. The second column in each pair describes the *final* concentration of that material. If the material is dsDNA (has non-zero length), the units of final concentration are assumed to be nM. Otherwise, the units of final concentration are the same as the units of concentration used for that material in the source plate.

Note that unlike the other two experimental settings of EchoRun, `build_picklist_from_association_spreadsheet` does *not* require that its EchoRun object be associated with a source plate file or SourcePlate object prior to the function being called -- a new SourcePlate object will be manufactured from the input spreadsheet when you call `load_source_plate`. 

# Tweaking Settings

#### To change the reaction volume:

By default, the Echo package assumes that you want to make 10 uL reactions. If you want to use a different volume, you'll need to change this in your `EchoRun` object. *Note that the `EchoRun`'s reaction volume has units of nanoliters!*

In [68]:
# You can set reaction volume in the EchoRun's constructor...
example_echo_calculator = mt_echo.EchoRun(rxn_vol = 20000) 
#... or dynamically.
example_echo_calculator.rxn_vol = 10.5 * 1e3 # Volume IN nL!!!

Make sure to change the `EchoRun`'s reaction volume *before* running `build_picklist_from_association_spreadsheet` or `build_dilution_series`. You almost certainly shouldn't do this at all when using `build_picklist_from_txtl_setup_csvs`, because that function will automatically extract a reaction volume from the setup spreadsheet.

If you use a `MixtureMaterial` or `TXTLMasterMix`, you should also set that object's reaction volume, in the same way as the `EchoRun`. If you try to use a mixture with an `EchoRun` with a different reaction volume, the `EchoRun`'s reaction volume will take precedence, and you will get a warning:

In [69]:
my_mixture = mt_echo.MixtureMaterial("Master Mix", rxn_vol = 15000)
example_echo_calculator.add_material_to_well(my_mixture, "A1", 0.5)

#### To change the master mix composition/extract fraction:

This is really only relevant for manual experimental setups -- TX-TL setup from a spreadsheet pulls the extract fraction from the spreadsheet, and the association spreadsheet method has no knowledge of TX-TL. 

When you construct an `EchoRun` object, you pass it a `TXTLMasterMix` object (or `None`). To change the extract fraction of an experiment, do this in the constructor of that `TXTLMasterMix` object. 

In [70]:
example_echo_calculator = mt_echo.EchoRun()
new_master_mix = mt_echo.TXTLMasterMix(extract_fraction = 0.40, 
                                       rxn_vol = example_echo_calculator.rxn_vol)
example_echo_calculator.dilution_series(gfp, atc, gfp_final_concentrations,
                                        atc_final_concentrations, "A1")
example_echo_calculator.add_material_to_all(new_master_mix)

You can also add arbitrary components to the master mix. For example, the following code ads the dye DFHBI-1T to every well at a final concentration of 10 µM, from a 2 mM stock:

In [71]:
example_plate = mt_echo.SourcePlate(os.path.join("tweaking", "example.dat"))
example_echo_calculator = mt_echo.EchoRun(plate = example_plate)
new_master_mix = mt_echo.TXTLMasterMix(rxn_vol = example_echo_calculator.rxn_vol)
dfhbi = mt_echo.EchoSourceMaterial("DFHBI-1T", 2000)
new_master_mix.add_material(dfhbi, 10)
example_echo_calculator.dilution_series(gfp, atc, gfp_final_concentrations,
                                              atc_final_concentrations, "A1")
example_echo_calculator.add_material_to_all(new_master_mix)

**To change buffer/extract aliquot size:**

Buffer and extract aliquot size are controlled by the `TXTLMasterMix` object. Like extract percentage, aliquot sizes can be changed in the `TXTLMasterMix`'s constructor.

Note that both aliquot sizes are in units of nL, not uL.

In [72]:
example_echo_calculator = mt_echo.EchoRun(plate = example_plate)

new_master_mix = mt_echo.TXTLMasterMix(extract_per_aliquot = 50000,
                                       buffer_per_aliquot = 70000,
                                       rxn_vol = example_echo_calculator.rxn_vol)
example_echo_calculator.dilution_series(gfp, atc, gfp_final_concentrations,
                                              atc_final_concentrations, "A1")
example_echo_calculator.add_material_to_all(new_master_mix)
# ...
# calculator_with_odd_destination.write_picklist(...)
#...

#### To change the source plate type/material type:

Source plate types and material types are set as optional arguments in the constructor of a SourcePlate object. The type and material of a source plate are both set by a string, which can be any string that the Echo Plate Reformat software will recognize.

In [73]:
plate_type = "384PP_AQ_BP" # This is actually the default plate value
retyped_example_plate = mt_echo.SourcePlate(filename = plate_file, SPtype = plate_type)
another_example_echo_calculator = mt_echo.EchoRun(plate = retyped_example_plate)

#### To change the source plate name:

Source plate names are set much like source plate types with the argument `SPname`. In addition, as a shorthand, you can set `SPname` to be a number N, in which case the plate will be named `Source[N]`. 

In [74]:
plate_name = "FirstPlate"
renamed_example_plate = mt_echo.SourcePlate(filename = plate_file, SPname = plate_name)
yet_another_example_echo_calculator = mt_echo.EchoRun(plate = renamed_example_plate)

**To change the destination plate type:**

Destination plate types are determined and stored directly in the EchoRun object. The destination plate type can be set by the optional argument `DPtype` in the constructor of an EchoRun object, or set manually any time before calling `write_picklist` on that EchoRun.

In [75]:
calculator_with_odd_destination = mt_echo.EchoRun(plate = example_plate, DPtype = "some_96_well_plate")
calculator_with_odd_destination.DPtype = "Nunc_384_black_glassbottom" # Just kidding!
# ...
# calculator_with_odd_destination.write_picklist(...)
#...

**To change dead volume and max volume**:

You probably shouldn't do this. If you absolutely must squeeze every last bit of efficiency out of your source wells (or if you want to use a low-dead-volume plate), you can set the `dead_volume` and `max_volume` variables, which are static variables in the murraylab_tools.echo package. If you change them, also make sure to set the static variable `usable_volume`, which defines the volume of material in a well that can actually be used by the Echo (this is normally calculated from `dead_volume` and `max_volume` at package import). Also, you should do this before running any experimental protocol function.

In [76]:
from murraylab_tools.echo.echo_functions import dead_volume, max_volume, usable_volume
dead_volume   = 10000 # Volume in nL!
max_volume    = 75000 # Volume in nL!
usable_volume = max_volume - dead_volume # Don't forget to re-calculate this!